# **Data Preprocessing**

In [1]:
import pandas as pd
import boto3
import awscli

In [6]:
# FETCH DATA FROM AWS

# Create the AWS S3 Client
s3_client = boto3.client('s3')
# Load the data in a dictionary 'response'
response = s3_client.get_object(Bucket = 'software-tools-ai', Key ='raw_data/listings.csv')
# Filter the content of the csv
csv = response['Body'].read()